<a href="https://colab.research.google.com/github/hemantkashniyal/EVA4/blob/master/notebooks/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
# import NN module 

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)     # input_channels: 1, output_channels: 32, num_kernels: 32, RF: 3, input_image_size: 30, output_image_size: 28 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)    # input_channels: 32, output_channels: 64, num_kernels: 64, RF: 5, input_image_size: 30, output_image_size: 28
        self.pool1 = nn.MaxPool2d(2, 2)                 # RF: 10, input_image_size: 28, output_image_size: 14
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)   # input_channels: 64, output_channels: 128, num_kernels: 128, RF: 12, input_image_size: 16, output_image_size: 14 
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)  # input_channels: 128, output_channels: 256, num_kernels: 256, RF: 14, input_image_size: 16, output_image_size: 14 
        self.pool2 = nn.MaxPool2d(2, 2)                 # RF: 28, input_image_size: 14, output_image_size: 7
        self.conv5 = nn.Conv2d(256, 512, 3)             # input_channels: 256, output_channels: 512, num_kernels: 512, RF: 28/30, input_image_size: 7, output_image_size: 5
        self.conv6 = nn.Conv2d(512, 1024, 3)            # input_channels: 512, output_channels: 1024, num_kernels: 1024, RF: 28/32, input_image_size: 5, output_image_size: 3
        self.conv7 = nn.Conv2d(1024, 10, 3)             # input_channels: 1024, output_channels: 10, num_kernels: 10, RF: 28/34, input_image_size: 3, output_image_size: 1 

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) 
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)
        # here we are stitching up network together, each input layer is followed by an activation function which amplifies value of each output, here we have used RELU as activation function.
        # after a couple of layers we are applying max pooling with size 2x2 and stride 2, this essentially doubles the receptive field of the layer.
        # At the end of these layers we are flattening the output to the desired 10 classes, and applying softmax function on the output. 

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))
# here we are checking if machine has a cuda enabled GPU or not.
# Net().to(device) in case of gpu moves the initialized parameter buffers to GPU for training.

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
# sets seed value for random number generation
batch_size = 128
# define image training batch size

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([ # chain the transformations on the input data
                        transforms.ToTensor(), # convert read data to tensor
                        transforms.Normalize((0.1307,), (0.3081,)) # normalize input tensor to given mean and standard deviation
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


0it [00:00, ?it/s]

9920512it [00:05, 1855052.80it/s]                             


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


32768it [00:00, 426578.66it/s]
  1%|          | 16384/1648877 [00:00<00:11, 142953.83it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


1654784it [00:00, 7098647.55it/s]                            
8192it [00:00, 178936.47it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train() # start training of model
    pbar = tqdm(train_loader) # define progress bar for training
    for batch_idx, (data, target) in enumerate(pbar): # run for each batch
        data, target = data.to(device), target.to(device) # data is grayscale image tensor, target is corresponding label
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target) # negative log likelyhood loss
        loss.backward()
        optimizer.step() # performs a single optimization
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:

model = Net().to(device) # Net().to(device) move model parameter buffers to GPU for training.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # use optimizer function stochastic gradient descent with learning rate of 0.01 and momentum of 0.9 

# learning rate:
# momentum:

for epoch in range(1, 2): # for 1 epoch do training and testing
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.16146743297576904 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.27it/s]



Test set: Average loss: 0.0575, Accuracy: 9801/10000 (98%)

